In [1]:
import os
import json
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib as mpl
import scipy.stats as st
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from tqdm import tqdm
from plotly import tools
from pandas import json_normalize
from plotly.offline import init_notebook_mode, iplot
import xgboost as xgb
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Create a list of colors 
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = 10, 6
pd.options.display.float_format = '{:,.0f}'.format
np.set_printoptions(formatter={'float': '{: 0.0f}'.format})

- Read csv and flatten json fields:   
https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

- Take string as an input and create a json object with the specified string as object name   
https://towardsdatascience.com/why-and-how-to-use-pandas-with-large-data-9594dda2ea4c

In [2]:

# Extract features from 'device', 'geoNetwork', 'totals', 'trafficSource’. 
json_columns = ['device', 'geoNetwork','totals', 'trafficSource']

def load_dataframe(csv_path):
    path = csv_path
    df = pd.read_csv(path, converters={column: json.loads for column in json_columns}, 
                     dtype={'fullVisitorId': 'str'})
   
    for column in json_columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df



In [5]:
train = load_dataframe('train.csv')
test = load_dataframe('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [4]:
# Store the dataframes as csv files
train.to_csv('train_df.csv')
test.to_csv('test_df.csv')

NameError: name 'train' is not defined

In [ ]:
# Read csv files
train = pd.read_csv('train_df.csv', dtype={'fullVisitorId': 'str'}, index_col=0)

test = pd.read_csv('test_df.csv', dtype={'fullVisitorId': 'str'}, index_col=0)

print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.head()

In [ ]:
test.info()

## 2. Data Preprocessing

 2.1 Remove Constant Features

Features that have only one unique value are often referred to as constant features or constant columns.   

These features do not provide any variability in the data, and as a result, they typically do not contribute    
useful information to the model. We might as well drop these features.

In [ ]:
train.columns

In [ ]:
# Keep 'totals_bounces' column
train['totals_bounces'].unique()

In [ ]:
train['totals_bounces'].fillna(0, inplace=True)
train['totals_bounces'].value_counts()

In [ ]:
# check unique values in train columns
train.nunique()

In [ ]:
# Identify constant features
constant_features = train.columns[train.nunique() == 1]

# Remove constant features
train.drop(columns=constant_features, inplace=True)
train.shape

In [ ]:
# Keep 'totals_bounces' column
test['totals_bounces'].unique()

In [ ]:
test['totals_bounces'].fillna(0, inplace=True)
test['totals_bounces'].value_counts()

In [ ]:
# check unique values in test columns
test.nunique()

In [ ]:
# Identify constant features
constant_features = test.columns[test.nunique() == 1]

# Remove constant features
test.drop(columns=constant_features, inplace=True)
test.shape

### 2.2 Find Percentage of missing values in the columns

- credits:   
https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset

In [ ]:
# train_df - Check the percentage of missing values in each column
percent_missing_train = train.isnull().sum() * 100 / len(train)

missing_value_train = pd.DataFrame({'percent_missing': percent_missing_train})
missing_value_train.sort_values(by='percent_missing', ascending=False, inplace=True)

# Columns with percent_missing > 0
missing_value_train_filtered = missing_value_train.loc[missing_value_train['percent_missing'] > 0]
missing_value_train_filtered

In [ ]:
# test_df - Check the percentage of missing values in each column
percent_missing_test = test.isnull().sum() * 100 / len(test)

missing_value_test = pd.DataFrame({#'column_name': test_df.columns,
                                 'percent_missing': percent_missing_test})
missing_value_test.sort_values(by='percent_missing', ascending=False, inplace=True)

# Columns with percent_missing > 0
missing_value_test_filtered = missing_value_test.loc[missing_value_test['percent_missing'] > 0]
missing_value_test_filtered

### 2.3 Columns to Drop

In [ ]:
train.columns

In [ ]:
cols_to_drop = ['geoNetwork_continent', 'geoNetwork_subContinent', 
                'geoNetwork_metro', 'geoNetwork_city', 'geoNetwork_networkDomain', 
                'trafficSource_campaign', 'trafficSource_keyword',
                'trafficSource_referralPath', 'trafficSource_adwordsClickInfo.page',
                'trafficSource_adwordsClickInfo.slot', 'trafficSource_adwordsClickInfo.gclId',
                'trafficSource_adContent', 'visitId', 'visitStartTime']

In [ ]:
df_train = train.copy()

df_train = df_train.drop(columns=cols_to_drop, axis=1)
df_train

In [ ]:
df_train.info()

In [ ]:
df_train.describe().T

In [ ]:
df_test = test.copy()

df_test = df_test.drop(columns=cols_to_drop, axis=1)
df_test

In [ ]:
df_test.info()

In [ ]:
df_test.describe().T

### 2.4 Impute Missing Values

In [ ]:
# numerical columns fillna
df_train['totals_pageviews'].fillna(0, inplace=True)
df_train['totals_transactionRevenue'].fillna(0, inplace=True)

df_test['totals_pageviews'].fillna(0, inplace=True)

In [ ]:
df_train.columns

In [ ]:
# Replace 'nan', 'NaN', '(not set)' in both dataframes
columns_to_process = ['device_browser', 'device_operatingSystem', 'trafficSource_source', 
                      'trafficSource_medium', 'trafficSource_adwordsClickInfo.adNetworkType']

values_to_replace = ['nan', 'NaN', '(not set)', np.nan]
replacement_value = 'others'

for column in columns_to_process:
    df_train[column].replace(values_to_replace, replacement_value, inplace=True)

for column in columns_to_process:
    df_test[column].replace(values_to_replace, replacement_value, inplace=True)

In [ ]:
df_train['trafficSource_adwordsClickInfo.adNetworkType'].unique()


In [ ]:
# Find the mismatched columns and the target variable
train_columns = set(df_train.columns)
test_columns = set(df_test.columns)
mismatched_columns = train_columns.symmetric_difference(test_columns)

print("Columns that don't match:")
print(mismatched_columns)

## 3. 'channelGrouping' & 'device' columns

In [ ]:
device_columns = ['channelGrouping', 'device_browser', 'device_operatingSystem', 
                  'device_isMobile', 'device_deviceCategory']

fig, ax = plt.subplots(figsize=(15, 8))

for i, col in enumerate(device_columns, 1):
    plt.subplot(2, 3, i)
    
    ax = sns.countplot(y=df_train[col], order=df_train[col].value_counts()[:10].index, )
    plt.title(f'Count of {col}')

    total = len(df_train[col])

    for c in ax.containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        ax.bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.tight_layout()
plt.show()

# Explicitly close the figure to remove overlapping axes
plt.close(fig)

### 3.1 Channels

In [ ]:
# Define the aggregation functions for each column
aggregations = {'fullVisitorId': 'count',
                'totals_pageviews': 'sum',
                'totals_transactionRevenue': 'sum'
}

In [ ]:
df_channels = df_train.groupby('channelGrouping').agg(aggregations
                                ).sort_values(by='totals_transactionRevenue', ascending=False)

df_channels

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plot each column in a separate subplot
for i, col in enumerate(df_channels.columns):
    df_channels[col].sort_values().plot(kind='barh', ax=axes[i], 
                                        color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_channels[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.tight_layout()
plt.show()

>**Observations:**

- Organic Search brings about the highest number of users & pageviews, but
- Referral brings in close to half of the revenues.
- Direct ranks the 3rd both in number of users & pageviews, yet second only to   
    Referral in terms of generating revenues as users know pretty much what they want.
- Social Media is not as effective in generating revenues as in generating the traffics.
- Paid Search has got what the advertisers have paid for: equal amount of traffics, pageviews   
    & revenues
- Affiliates are not that much effective compared with other channels above,

### 3.2 'device_browser'

In [ ]:
df_browser = df_train.groupby('device_browser').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False).head(10)

df_browser

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plot each column in a separate subplot
for i, col in enumerate(df_browser.columns):
    df_browser[col].sort_values().plot(kind='barh', ax=axes[i], color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_browser[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.suptitle('Browsers')
plt.tight_layout()
plt.show()

>**Observations:**
   
- Not suprising, Chrome is the powerhouse in all 3 fields, especially in    
    generating the revenues, close to 90 percent of the market share.
- Surprisingly, both Safari & Firefox outperforms Internet Explorer & Edge    
    in all 3 fields.


### 3.3 'device_operatingSystem'

In [ ]:
df_os = df_train.groupby('device_operatingSystem').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False).head(10)

df_os

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plot each column in a separate subplot
for i, col in enumerate(df_os.columns):
    df_os[col].sort_values().plot(kind='barh', ax=axes[i], color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_os[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.suptitle('Operating Systems')
plt.tight_layout()
plt.show()

### 3.4 'device_isMobile'

In [ ]:
df_mobile = df_train.groupby('device_isMobile').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False)

df_mobile

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Flatten the 2D array of axes
axes = axes.flatten()

# Plot each column in a separate subplot
for i, col in enumerate(df_mobile.columns):
    df_mobile[col].sort_values().plot(kind='barh', ax=axes[i], color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_mobile[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.suptitle('Mobile vs Non-Mobile')
plt.tight_layout()
plt.show()

### 3.5 'device.deviceCategory'

In [ ]:
df_category = df_train.groupby('device_deviceCategory').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False).head(10)

df_category

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plot each column in a separate subplot
for i, col in enumerate(df_category.columns):
    df_category[col].sort_values().plot(kind='barh', ax=axes[i], color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_category[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.suptitle('Device Category')
plt.tight_layout()
plt.show()

>**Observations:**

desktop & non-mobile devices
- still dominate in bring in traffics, pageviews & revenues

In [ ]:
device_cols = ["device_browser", "device_deviceCategory", "device_operatingSystem"]

colors = ["#d6a5ff", "#fca6da", "#f4d39c", "#a9fcca"]
traces = []
for i, col in enumerate(device_cols):
    t = train[col].value_counts()
    traces.append(go.Bar(marker=dict(color=colors[i]),orientation="h", y = t.index[:15][::-1], x = t.values[:15][::-1]))

fig = tools.make_subplots(rows=1, cols=3, subplot_titles=["Visits: Category", "Visits: Browser","Visits: OS"], print_grid=False)
fig.append_trace(traces[1], 1, 1)
fig.append_trace(traces[0], 1, 2)
fig.append_trace(traces[2], 1, 3)

fig['layout'].update(height=400, showlegend=False, title="Visits by Device Attributes")
iplot(fig)

## convert transaction revenue to float
train["totals_transactionRevenue"] = train["totals_transactionRevenue"].astype('float')

device_cols = ["device_browser", "device_deviceCategory", "device_operatingSystem"]

fig = tools.make_subplots(rows=1, cols=3, subplot_titles=["Mean Revenue: Category", "Mean Revenue: Browser","Mean Revenue: OS"], print_grid=False)

colors = ["red", "green", "purple"]
trs = []
for i, col in enumerate(device_cols):
    tmp = train.groupby(col).agg({"totals_transactionRevenue": "mean"}).reset_index().rename(columns={"totals_transactionRevenue" : "Mean Revenue"})
    tmp = tmp.dropna().sort_values("Mean Revenue", ascending = False)
    tr = go.Bar(x = tmp["Mean Revenue"][::-1], orientation="h", marker=dict(opacity=0.5, color=colors[i]), y = tmp[col][::-1])
    trs.append(tr)

fig.append_trace(trs[1], 1, 1)
fig.append_trace(trs[0], 1, 2)
fig.append_trace(trs[2], 1, 3)
fig['layout'].update(height=400, showlegend=False, title="Mean Revenue by Device Attributes")
iplot(fig)

###  GeoNetwork Attributes

In [ ]:
geo_cols = ['geoNetwork_city', 'geoNetwork_continent','geoNetwork_country',
            'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region','geoNetwork_subContinent']
geo_cols = ['geoNetwork_continent','geoNetwork_subContinent']

colors = ["#d6a5ff", "#fca6da"]
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=["Visits : GeoNetwork Continent", "Visits : GeoNetwork subContinent"], print_grid=False)
trs = []
for i,col in enumerate(geo_cols):
    t = train[col].value_counts()
    tr = go.Bar(x = t.index[:20], marker=dict(color=colors[i]), y = t.values[:20])
    trs.append(tr)

fig.append_trace(trs[0], 1, 1)
fig.append_trace(trs[1], 1, 2)
fig['layout'].update(height=400, margin=dict(b=150), showlegend=False)
iplot(fig)




geo_cols = ['geoNetwork_continent','geoNetwork_subContinent']
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=["Mean Revenue: Continent", "Mean Revenue: SubContinent"], print_grid=False)

colors = ["blue", "orange"]
trs = []
for i, col in enumerate(geo_cols):
    tmp = train.groupby(col).agg({"totals_transactionRevenue": "mean"}).reset_index().rename(columns={"totals_transactionRevenue" : "Mean Revenue"})
    tmp = tmp.dropna().sort_values("Mean Revenue", ascending = False)
    tr = go.Bar(y = tmp["Mean Revenue"], orientation="v", marker=dict(opacity=0.5, color=colors[i]), x= tmp[col])
    trs.append(tr)

fig.append_trace(trs[0], 1, 1)
fig.append_trace(trs[1], 1, 2)
fig['layout'].update(height=450, margin=dict(b=200), showlegend=False)
iplot(fig)

## 4. 'geoNetwork' columns

In [ ]:
geo_columns = ['geoNetwork_continent', 'geoNetwork_subContinent', 'geoNetwork_country', 
               'geoNetwork_region', 'geoNetwork_metro', 'geoNetwork_city', 'geoNetwork_networkDomain']

fig, ax = plt.subplots(figsize=(20, 8))

for i, col in enumerate(geo_columns, 1):
    plt.subplot(2, 4, i)
    
    ax = sns.countplot(y=train[col], order=train[col].value_counts()[:10].index,  )
    plt.title(f'Count of {col}')    
    
    total = len(train[col])

    for c in ax.containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        ax.bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.tight_layout()
plt.show()

# Explicitly close the figure to remove overlapping axes
plt.close(fig)

### 4.1 Global Analysis

In [ ]:
df_country = df_train.groupby('geoNetwork_country').agg(aggregations
                    ).sort_values(by=['totals_transactionRevenue'], 
                                  ascending=False).reset_index()
df_country.head(10)

In [ ]:
# Create basic choropleth map
fig = px.choropleth(df_country, locationmode='country names', locations='geoNetwork_country', 
                    color=df_country['totals_transactionRevenue'], color_continuous_scale='Mint', 
                    hover_name='geoNetwork_country',  hover_data={'totals_transactionRevenue':':,.0f'},
                    title='Revenue by Country')

fig.update_layout(width=800, height=400, autosize=False, 
                  margin=dict(l=30, r=30, t=50, b=50, pad=10))

fig.show(renderer='notebook')
# fig.write_html("interactive_map.html")

### 4.2 US Analysis

In [ ]:
df_us = df_train[df_train['geoNetwork_country'] == 'United States'].reset_index(drop=True)
df_us

In [ ]:
# Add a column with state abbreviations
state_abbreviations = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
                       'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
                       'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
                       'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
                       'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
                       'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
                       'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
                       'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
                       'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
                       'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
                       'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC' }

df_us['stateCode'] = df_us['geoNetwork_region'].map(state_abbreviations)
df_us.head()

In [ ]:
df_states = df_us.groupby(['geoNetwork_region', 'stateCode']).agg(aggregations).sort_values(
                                            by='totals_transactionRevenue', ascending=False).reset_index()
df_states.head(10)

In [ ]:
# Create basic choropleth map
fig = px.choropleth(df_states, locationmode='USA-states', locations='stateCode',
                    color=df_states['totals_transactionRevenue'], hover_name='geoNetwork_region', 
                    hover_data={'totals_transactionRevenue':':,.0f'}, 
                    title='Revenue by State', color_continuous_scale='Mint')
                  
fig.update_layout(width=800, height=400, autosize=False, geo_scope='usa',
                  margin=dict(l=30, r=30, t=50, b=50, pad=10))

# For interactive display in a Jupyter Notebook:
fig.show(renderer='notebook')
# fig.show()

## 6. 'trafficSource' Columns

In [ ]:
traffic_columns = ['trafficSource_campaign', 'trafficSource_source', 'trafficSource_medium', 
                   'trafficSource_keyword', 'trafficSource_referralPath', 
                   'trafficSource_adwordsClickInfo.page', 'trafficSource_adwordsClickInfo.slot', 
                   'trafficSource_adwordsClickInfo.adNetworkType', 'trafficSource_adContent']

fig, ax = plt.subplots(figsize=(18, 12))

for i, col in enumerate(traffic_columns, 1):
    plt.subplot(3, 3, i)
    
    # Exclude 'others' & '(not set)' from the specified column
    filtered_df = train[(train[col] != 'others') & 
                        (train[col] != '(not provided)') & 
                        (train[col] != '(not set)')]
    
    ax = sns.countplot(y=filtered_df[col], order=filtered_df[col].value_counts()[:7].index)
    plt.title(f'Count of {col}')

    total = len(train[col])

    for c in ax.containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        ax.bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.tight_layout()
plt.show()

# Explicitly close the figure to remove overlapping axes
plt.close(fig)

In this section, we have excluded 'others' & '(not set)' from the specified columns for they comprise    
an overwhelming large part of the data in this section.

We are only interested in the foloowing features:   

- 'trafficSource.source'    
- 'trafficSource.medium'   

### 6.1 'trafficSource.source'

- The source of the traffic source. Could be the name of the search engine, etc.
- Top 10 sources are:   
    'google', 'youtube.com', '(direct)', 'mall.googleplex.com', 'Partners',   
    'analytics.google.com', 'dfa', 'google.com', 'm.facebook.com', 'baidu'.      
    google: 44% of the trafficsource source

In [ ]:
df_source = df_train.groupby('trafficSource_source').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False).head(10)

df_source

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plot each column in a separate subplot
for i, col in enumerate(df_source.columns):
    df_source[col].sort_values().plot(kind='barh', ax=axes[i], color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_source[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.suptitle('TrafficSource Source')
plt.tight_layout()
plt.show()

>**Observations:**

'trafficSource_source'    
    
- 'google' brings in most of the users & pageviews, while 'mall.googleplex.com' generates most of the revenues.
- '(direct)' ranks second in all 3 fields. No wonder again, users know what they want & type the urls in the browser.
- 'dfa': DFA source refers to DoubleClick For Advertisers, which is the old brand name for Google Marketing Platform. 

### 6.2 'trafficSource_medium'

- The medium of the traffic source. "organic", "cpc", "referral", etc,   
- Top 6 sources are:   
    'organic', 'referral', '(none)', 'cpc', 'affiliate', 'cpm', etc.   
    organic: 42% of the traffucesource medium

In [ ]:
df_medium = df_train.groupby('trafficSource_medium').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False).head(10)

df_medium

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plot each column in a separate subplot
for i, col in enumerate(df_medium.columns):
    df_medium[col].sort_values().plot(kind='barh', ax=axes[i], color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_medium[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.suptitle('TrafficSource Medium')
plt.tight_layout()
plt.show()

>**Observations:**

- "organic" bring about most of the sessions & pageviews, it is   
    "referral" that generates most of the revenues. 
- "cpc": Cost Per Click. Plays an important role in all 3 fields.
- '(none)': means Analytics doesn't have any referral information for these users.

#### 6.3 Source & Medium

In [ ]:
df_source_medium = df_train.groupby(['trafficSource_source', 'trafficSource_medium']).agg(aggregations)

# df_source_medium.sort_values(by='revenue', ascending=False).head(10)
df_source_medium

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
# Flatten the 2D array of axes
# axes = axes.flatten()

# Plot each column in a separate subplot
for i, col in enumerate(df_source_medium.columns):
    df_source_medium[col].sort_values(ascending=False)[:10].plot(kind='barh', ax=axes[i], color=colors)
    axes[i].set_title(col)
    axes[i].set_xlabel('Count')

    total = df_source_medium[col].sum()

    for c in axes[i].containers:
        percentages = [f'{(v / total) * 100:.2f}%' for v in c.datavalues]
        axes[i].bar_label(c, labels=percentages, fmt='%s', padding=1)

plt.suptitle('TrafficSource Source & Medium')
plt.tight_layout()
plt.show()

>**Observations:**

- 'google organic' generates most of the traffics, the rererrals from    
    'mall.googleplex.com' generates most of the revenue.
- 'direct none': again, Analytics does not have any information about users,   
    yet are dominant in all 3 fields on average after 'google organic'.

## 7. Revenue = 0 vs Revenue > 0

In [ ]:
# revenue = 0 transactions
df_revenue_zero = df_train[df_train['totals_transactionRevenue'] == 0]
df_revenue_zero.head()

In [ ]:
# revenue > 0 transactions
df_revenue_nonzero = df_train[df_train['totals_transactionRevenue'] > 0].reset_index(drop=True)
df_revenue_nonzero.head()

In [ ]:
non_zero_transact_pct = len(df_revenue_nonzero) / len(df_train) * 100
print(f'Non-Zero Transaction Percentage: {non_zero_transact_pct:.02f}%')

zero_transact_pct = len(df_revenue_zero) / len(df_train) * 100
print(f'Zero Transaction Percentage: {zero_transact_pct:.02f}%')

>***Observations:***

- 98.73% of the traffics (sessions, pageviews) does not generate revenues at all.

In [ ]:
# Filter the DataFrame for zero revenue
# df_revenue_zero = df_train[df_train['totals.transactionRevenue'] == 0]

# List of columns to loop through
columns_to_loop = ['channelGrouping', 'trafficSource_source', 'trafficSource_medium']

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Loop through each column and subplot
for i, column in enumerate(columns_to_loop):
    # Create the aggregated DataFrame
    df_column_revenue_zero = df_revenue_zero.groupby(column).agg(
        count=(column, 'count') ).sort_values(by='count', ascending=False).head(10)

    # Calculate the percentage
    total_count = df_column_revenue_zero['count'].sum()
    df_column_revenue_zero['percentage'] = (df_column_revenue_zero['count'] / total_count) * 100

    # Plot the horizontal bar chart in the current subplot
    bar_chart = df_column_revenue_zero['count'].plot(kind='barh', ax=axs[i], color=colors)

    # Add labels and title
    axs[i].set_xlabel(f'Count of Zero Transactions ({column})')
    axs[i].set_ylabel(column)
    axs[i].set_title(f'Top 10 {column} with Zero Transactions')

    # Display percentages on the right side of the bars
    for j, patch in enumerate(bar_chart.patches):
        width = patch.get_width()
        percentage = df_column_revenue_zero['percentage'].iloc[j]
        axs[i].text(width + 1000, patch.get_y() + patch.get_height() / 2,
                    f'{percentage:.2f}%', ha='left', va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Filter the DataFrame for non-zero revenue
# df_revenue_nonzero = df_train[df_train['totals_transactionRevenue'] > 0]

# List of columns to loop through
columns_to_loop = ['channelGrouping', 'trafficSource_source', 'trafficSource_medium']

# Plot the bar chart
fig, ax = plt.subplots(3, 1, figsize=(10, 9))

# Loop through each column
for i, col in enumerate(columns_to_loop):
    # Create the aggregated DataFrame
    df_column_revenue_nonzero = df_revenue_nonzero.groupby(col).agg(
        count=(col, 'count')
    ).sort_values(by='count', ascending=False).head(10)

    # Calculate the percentage
    total_count = df_column_revenue_nonzero['count'].sum()
    df_column_revenue_nonzero['percentage'] = (df_column_revenue_nonzero['count'] / total_count) * 100

    bar_chart = df_column_revenue_nonzero['count'].plot(kind='bar', rot=0, ax=ax[i], color=colors)

    # Add labels and title
    ax[i].set_ylabel(f'Count of Non-Zero Transactions ({col})')
    ax[i].set_xlabel(col)
    ax[i].set_title(f'Top 10 {col} with Non-Zero Transactions')

    # Display percentages on top of the bars
    for j, patch in enumerate(bar_chart.patches):
        height = patch.get_height()
        percentage = df_column_revenue_nonzero['percentage'].iloc[j]
        ax[i].text(patch.get_x() + patch.get_width() / 2,
                height + 1000, f'{percentage:.2f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Filter the DataFrame for zero revenue
# df_revenue_zero = df_train[df_train['totals_transactionRevenue'] == 0]

# List of columns to loop through
columns_to_loop = ['channelGrouping', 'trafficSource_source', 'trafficSource_medium']

# Loop through each column
for col in columns_to_loop:
    # Create the aggregated DataFrame
    df_column_revenue_zero = df_revenue_zero.groupby(col).agg(
        count=(col, 'count')
    ).sort_values(by='count', ascending=False).head(10)

    # Calculate the percentage
    total_count = df_column_revenue_zero['count'].sum()
    df_column_revenue_zero['percentage'] = (df_column_revenue_zero['count'] / total_count) * 100

    # Plot the bar chart
    fig, ax = plt.subplots(figsize=(12, 4))
    # Flatten the 2D array of axes
    # axes = axes.flatten()

    bar_chart = df_column_revenue_zero['count'].plot(kind='bar', rot=0, ax=ax, color=colors)

    # Add labels and title
    ax.set_ylabel(f'Count of Zero Transactions ({col})')
    ax.set_xlabel(col)
    ax.set_title(f'Top 10 {col} with Zero Transactions')

    # Display percentages on top of the bars
    for i, patch in enumerate(bar_chart.patches):
        height = patch.get_height()
        percentage = df_column_revenue_zero['percentage'].iloc[i]
        ax.text(patch.get_x() + patch.get_width() / 2,
                height + 1000, f'{percentage:.2f}%', ha='center', va='bottom', fontsize=12)

    plt.tight_layout()
    plt.show()

>**Observations:**

- Most of the zero-revenue transacations are from 'organic search' or 'search google'
- while most of the non-zero transactions are from 'mall.googleplex.com' & 'referral',    
    '(direct)' plays an important role in bringing about traffics & revenues.

## 8. User & Session Analysis

### 8.1 User Analysis

In [ ]:
df_visitors = df_train.groupby('fullVisitorId').agg(aggregations).sort_values(
                    by='totals_transactionRevenue', ascending=False )

df_visitors.drop('fullVisitorId', axis=1).head(10)

In [ ]:
# Users with non-zero revenue transactions
df_users_revenue = df_revenue_nonzero.groupby('fullVisitorId').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False)

df_users_revenue.drop('fullVisitorId', axis=1)

In [ ]:
# Distribution of non-zero revenue transactions
non_zero = df_train[df_train['totals_transactionRevenue'] > 0] \
                 ['totals_transactionRevenue'].reset_index(drop=True)
    
fig, ax = plt.subplots(figsize=(10, 5))

sns.histplot(non_zero, kde=True, bins=30, log_scale=True)
ticks_x = ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000))
ax.xaxis.set_major_formatter(ticks_x)

plt.title('Distribution of Non-Zero Revenue Transactions')
plt.xlabel('Transaction Revenue in Thousands')

plt.show()

In [ ]:
# Users with zero revenue transactions
df_user_revenue_zero = df_revenue_zero.groupby('fullVisitorId').agg(aggregations
                ).sort_values(by='totals_transactionRevenue', ascending=False)

df_user_revenue_zero.drop('fullVisitorId', axis=1)

In [ ]:
df_user_revenue_zero_pct = round(len(df_user_revenue_zero['totals_transactionRevenue']) / 
                                 len(df_train['totals_transactionRevenue']) * 100, 2)
print(f'Percentage of users witn zero-revenue trasactions:\n {df_user_revenue_zero_pct}%')

### 8.2 Sessions

In [ ]:
# Non-zero revenue sessions
df_session_revenue = df_revenue_nonzero.groupby('sessionId').agg(
                count=('sessionId', 'count'),
                pageviews=('totals_pageviews', 'sum'),
                revenue=('totals_transactionRevenue', 'sum')
                ).sort_values(by='revenue', ascending=False)

df_session_revenue

In [ ]:
# Sessions with zero revenue transactions
df_session_revenue_zero = df_revenue_zero.groupby('sessionId').agg(
                sessions=('sessionId', 'count'),
                pageviews=('totals_pageviews', 'sum'),
                revenue=('totals_transactionRevenue', 'sum')
                ).sort_values(by='revenue', ascending=False)

df_session_revenue_zero

In [ ]:
df_session_revenue_zero_pct = round(len(df_session_revenue_zero['revenue']) / 
                                    len(df_train['totals_transactionRevenue']) * 100, 2)

print(f'Percentage of sessions witn zero-revenue trasactions:\n {df_session_revenue_zero_pct}%')

## 9. Time Analysis

### 9.1 Extract 'Year', 'Month', 'Month_Num' & 'Day' from 'date' column

In [ ]:
# Extract 'Year', 'Month', 'Month_Num' & 'Day' from 'date'
def create_features(data):
    """
    Create time series features based on time series index.
    """

    # data = data.copy()
    
    # Convert 'date' column to datetime series 
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')

    # Extract 'Year', 'Month', 'Month_Num' & 'Day' from 'Order Date'
    data['Year'] = data.date.dt.year
    data['Month'] = data.date.dt.strftime('%b') # abbrev month name
    data['Month_Num'] = data.date.dt.month
    data['Day'] = data.date.dt.day

    # Extract 'Weekday' & 'Week Day Num'
    data['Weekday'] = data['date'].dt.strftime('%a')
    data['Weekday_Num'] = data.date.dt.day_of_week + 1 # Week start on Monday as 0, so + 1

    return data

In [ ]:
df_train = df_train.copy()

create_features(df_train)

df_train = df_train[df_train['date'] != '2017-08-01']
df_train.tail()

### 9.2 Daily Transactions

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
df_daily = df_train.groupby('date').agg(aggregations)
df_daily

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(15, 6), sharex=True)

ticks_y = ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000000))

sns.lineplot(data=df_daily, x='date', y='totals_transactionRevenue', color='g', label='Revenue in Millions', ax=ax[0])
# plt.gca(ax=ax[0]).yaxis.set_major_formatter(ticks_y)
ax[0].yaxis.set_major_formatter(ticks_y)

# sns.lineplot(data=df_daily, x='date', y='totals.hits', color='b', label='Hits', ax=ax[1])
sns.lineplot(data=df_daily, x='date', y='totals_pageviews', color='b', label='Pageviews', ax=ax[1])

sns.lineplot(data=df_daily, x='date', y='fullVisitorId', color='k', label='Visitors', ax=ax[2])

plt.legend()
plt.suptitle('Daily Visitors, Pageviews & Revenue')
plt.tight_layout()
plt.show()

### 9.3 Weekly Anslysis

In [ ]:
df_week = df_train.groupby(['Weekday', 'Weekday_Num']).agg(aggregations)

df_week.sort_values(by='Weekday_Num', inplace=True)
df_week 

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 4), sharex=False)

sns.lineplot(data=df_week, x='Weekday', y='fullVisitorId', color='k', label='Visitors', ax=ax[0])
sns.lineplot(data=df_week, x='Weekday', y='totals_pageviews', color='b', label='Pageviews', ax=ax[1])

ticks_y = ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000000))
sns.lineplot(data=df_week, x='Weekday', y='totals_transactionRevenue', color='g', label='Revenuev in Millions', ax=ax[2])
ax[2].yaxis.set_major_formatter(ticks_y)

plt.legend()
plt.suptitle('Weekly Visitors, Pageview & Revenue')

plt.tight_layout()
plt.show()

### 9.4 Monthly Analysis

In [ ]:
df_train_dates = df_train.set_index(df_train['date']).drop('date', axis=1)

# months = df_train_dates['Month'].unique()
months = ['Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul']

# Set up subplots
fig, axes = plt.subplots(nrows=len(months), ncols=1, 
                         figsize=(10, 3*len(months)))

# fig.suptitle('Monthly Revenues in Millions')

# Plot for each year
for i, month in enumerate(months):
    df_monthly = df_train_dates[df_train_dates['Month'] == month]
    df_monthly_rev = df_monthly['totals_transactionRevenue']

    # Set subplot title
    axes[i].set_title(f'Monthly Revenues in Millions - {month}')

    ticks_y = ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000000))

    # Plot on the i-th subplot
    # df_month_rev.plot(kind='line', ax=axes[i], rot=0)
    sns.lineplot(data=df_monthly_rev, ax=axes[i], legend=True)
    axes[i].yaxis.set_major_formatter(ticks_y)

    # Set y-axis scale
    # axes[i].set_yscale('log') 

plt.tight_layout()
plt.show()

In [ ]:
df_month = df_train.groupby(['Year', 'Month', 'Month_Num']).agg(aggregations)
df_month = df_month[df_month.index != '2017-08-01']

df_month.sort_values(by=['Year', 'Month_Num'], inplace=True)
df_month = df_month.reset_index()

df_month

## 10. Machine Learning Model

In [ ]:
import lightgbm as lgb 
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

### 10.1 Data Transformation

In [ ]:
### Create calendar columns in test dataset
df_test = df_test.copy()

create_features(df_test)
df_test.tail()

In [ ]:
train_set = df_train.drop(columns=['Year', 'Month', 'Weekday'], axis=1).reset_index(drop=True)
test_set = df_test.drop(columns=['Year', 'Month', 'Weekday'], axis=1).reset_index(drop=True)

print(train_set.shape, test_set.shape)

In [ ]:
# numeric_columns = train_set.select_dtypes(include='number').columns
numeric_cols = [c for c in train_set.columns if c.startswith('total') and c != 'totals_transactionRevenue']
numeric_cols

In [ ]:
for col in numeric_cols:

    # Check for and handle missing values
    train_set[col].fillna(0, inplace=True) 
    test_set[col].fillna(0, inplace=True)

    sc = StandardScaler()
    # Fit the StandardScaler on the combined data (train and test)
    sc.fit(pd.concat([train_set[[col]], test_set[[col]]], axis=0, ignore_index=True))

    # Transform the values in both train and test sets
    train_set[col] = sc.transform(train_set[[col]])
    test_set[col] = sc.transform(test_set[[col]])

In [ ]:
# categorical_columns = train_set.select_dtypes(include='object').columns

## exclude columns for normalization or label-encoding
cols_excluded = ['date', 'visitNumber']

categorical_cols = [c for c in train_set.columns if c not in cols_excluded and not c.startswith('total')]
categorical_cols

In [ ]:
# Convert date to UNIX timestamp
train_set['date'] = train['date'].astype(int)
test_set['date'] = test['date'].astype(int)

In [ ]:
for col in categorical_cols:

    le = LabelEncoder()
    # Fit the LabelEncoder on the combined data (train and test)
    le.fit(pd.concat([train_set[col], test_set[col]], axis=0, ignore_index=True))
    
    # Transform the values in both train and test sets
    train_set[col] = le.transform(train_set[col])
    test_set[col] = le.transform(test_set[col])

In [ ]:
X = train_set.drop('totals_transactionRevenue', axis=1)
y = train_set['totals_transactionRevenue']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
params = {'num_leaves': 38,              
          'min_data_in_leaf': 50,
          'objective': 'regression',     
          'max_depth': -1,                
          'learning_rate': 0.1,           
          'verbose': -1,
          'predict_disable_shape_check': [True]
          }

train_data = lgb.Dataset(X_train, y_train)
test_data = lgb.Dataset(X_test, y_test)

model = lgb.train(params,
                train_data,
                20000,
                valid_sets=[train_data, test_data],
                callbacks=[lgb.early_stopping(stopping_rounds=500), lgb.log_evaluation(50)])

In [ ]:
preds = model.predict(test_set, num_iteration=model.best_iteration)
preds

In [ ]:
test_set["PredictedRevenue"] = preds
test_set

In [ ]:
lgb_params = {'objective': 'regression',
              'metric': 'rmse',
              'num_leaves': 50,
              'learning_rate': 0.01,
              'bagging_fraction': 0.75,
              'feature_fraction': 0.8,
              'bagging_frequency': 5,
              'force_row_wise': True,  
              'verbose': -1,
              'predict_disable_shape_check': True  
}
    
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_test, label=y_test)
model_1 = lgb.train(lgb_params, lgb_train, 
                  num_boost_round=700, 
                  valid_sets=[lgb_val], 
                  callbacks=[lgb.early_stopping(stopping_rounds=500), 
                             lgb.log_evaluation(50), ])

In [ ]:
preds_1 = model_1.predict(test_set.loc[:, test_set.columns != 'PredictedRevenue'], 
                          num_iteration=model.best_iteration, )
preds_1

In [ ]:
test_set["PredictedRevenue"] = preds_1
test_set

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgb_params = {'num_leaves' : 50, 'learning_rate' : 0.01, 
              'bagging_fraction' : 0.75, 'feature_fraction' : 0.8, 
              'bagging_frequency' : 5, 'force_row_wise': [True], 
              'verbose': -1,}
    
model_2 = LGBMRegressor(**lgb_params, n_estimators=700)
model_2.fit(X_train, y_train, 
          eval_set=[(X_test, y_test)])

In [ ]:
preds_2 = model_2.predict(test_set.loc[:, test_set.columns != 'PredictedRevenue'], 
                          num_iteration=model.best_iteration, )
preds_2

In [ ]:
test_set["PredictedRevenue"] = preds_2
test_set

In [ ]:
pip install Flask pandas xgboost

In [ ]:
import joblib
# Assuming `model` is your trained XGBoost model
joblib.dump(model, 'xgboost_model.pkl')